Ideally we would want each token that python requires to be it's own token. This is not strictly necessary but I believe it will help since we will be training quite small models. Will probably also reduce the amount of synxtax errors. 

In [56]:
import tokenize

tokenize.EQUAL, ord("=") # not ascii

(22, 61)

In [91]:
from io import BytesIO


python_code = """
# sum from 1 to 9
accumulator = 0 # accumulates the sum
for i in range(10):
    accumulator += 1
"""

list(x.string for x in tokenize.tokenize(BytesIO(python_code.encode("utf-8")).readline)) # don't know why utf-8 is there

['utf-8',
 '\n',
 '# sum from 1 to 9',
 '\n',
 'accumulator',
 '=',
 '0',
 '# accumulates the sum',
 '\n',
 'for',
 'i',
 'in',
 'range',
 '(',
 '10',
 ')',
 ':',
 '\n',
 '    ',
 'accumulator',
 '+=',
 '1',
 '\n',
 '',
 '']